# Rebuild Transit Example

In [1]:
import sys
sys.path.append("/home/ubuntu/git/aequilibrae")  # path to AequilibraE

In [2]:
from uuid import uuid4
from os import remove
from os.path import join, exists
from tempfile import gettempdir
from zipfile import ZipFile
from pathlib import Path

from aequilibrae.transit import Transit
from aequilibrae.utils.create_example import create_example


/home/ubuntu/git/aequilibrae/aequilibrae/utils/python_signal.py:17: UserWarning: No progress bars will be shown. Please install tqdm to see them
  warnings.warn("No progress bars will be shown. Please install tqdm to see them")


In [3]:
def rebuid_coquimbo_example(dest_folder):
    """
    dest_folder: where the zip file with the complete model is stored
    gtfs_path: path to gtfs_file
    """
    fldr = join(gettempdir(), uuid4().hex)
    project = create_example(fldr, "coquimbo")
    gtfs_path = join(fldr, "gtfs_coquimbo.zip")

    remove(join(fldr, "public_transport.sqlite"))

    data = Transit(project)

    transit = data.new_gtfs_builder(agency="Lisanco", file_path=gtfs_path)

    transit.load_date("2016-04-13")

    transit.set_allow_map_match(True)
    transit.map_match()

    transit.save_to_disk()

    cursor = Transit(project).pt_con.cursor()
    cursor.execute('VACUUM;')

    with ZipFile(f'{dest_folder}/coquimbo.zip', 'w') as zip_object:
        zip_object.write(f"{fldr}/project_database.sqlite", "project_database.sqlite")
        zip_object.write(f"{fldr}/public_transport.sqlite", "public_transport.sqlite")
        zip_object.write(f"{fldr}/gtfs_coquimbo.zip", "gtfs_coquimbo.zip")
        zip_object.write(f"{fldr}/parameters.yml", "parameters.yml")

    if exists(f'{dest_folder}/coquimbo.zip'):
        print("ZIP file created!")


In [4]:
dest_folder = Path("/home/ubuntu/git/aequilibrae/aequilibrae/reference_files")

In [5]:
rebuid_coquimbo_example(dest_folder)

ZIP file created!
